<a href="https://colab.research.google.com/github/KaichengDING/Triple-Defense/blob/main/idl_project_creating_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

import numpy as np

import PIL

import sys
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

from torch.utils import data

import matplotlib.pyplot as plt
import time
import logging
import datetime

cuda = torch.cuda.is_available()
cuda

True

In [8]:
class AddGaussianNoise(object):
    def __init__(self, mean=0.0, std=1.0, dropout=0.5):
        self.std = std
        self.mean = mean
        self.dropout = dropout
        
    def __call__(self, tensor):
      mask = torch.rand(tensor.size()) >= self.dropout
      noise = torch.randn(tensor.size()) * self.std + self.mean
      noise = noise * mask
      res = tensor + noise
      return torch.clamp(input=res, min=0.0, max=1.0)
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [9]:
def train_epoch(model, train_loader, criterion, optimizer):
    model.train()

    running_loss = 0.0
    total_predictions = 0.0
    correct_predictions = 0.0
    
    for batch_idx, (data, target) in enumerate(train_loader):   
        optimizer.zero_grad()   # .backward() accumulates gradients
        data = data.to(device)
        target = target.to(device) # all data & model on same device
        
        with torch.cuda.amp.autocast():
          outputs = model(data)
          loss = criterion(outputs, target)
          running_loss += loss.item()

          _, predicted = torch.max(outputs.data, 1)
          total_predictions += target.size(0)
          correct_predictions += (predicted == target).sum().item()
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()


        del data
        del target
    
    
    running_loss /= len(train_loader)
    acc = (correct_predictions/total_predictions)*100.0
    logging.info('Training Loss: {}'.format(running_loss))
    logging.info('Training Accuracy: {}%'.format(acc))
    return running_loss


def test_model(model, test_loader, criterion):
    with torch.no_grad():
        model.eval()

        running_loss = 0.0
        total_predictions = 0.0
        correct_predictions = 0.0

        for batch_idx, (data, target) in enumerate(test_loader): 
            data = data.to(device)
            target = target.to(device)

            with torch.cuda.amp.autocast():
              outputs = model(data)

              _, predicted = torch.max(outputs.data, 1)
              total_predictions += target.size(0)
              correct_predictions += (predicted == target).sum().item()

              loss = criterion(outputs, target).detach()
              running_loss += loss.item()

            del data
            del target

        running_loss /= len(test_loader)
        acc = (correct_predictions/total_predictions)*100.0
        logging.info('Testing Loss: {}'.format(running_loss))
        logging.info('Testing Accuracy: {}%'.format(acc))
        return running_loss, acc

def init_weights(m):
    if type(m) == nn.Conv2d or type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight.data)

In [10]:
# configure logging
logger = logging.getLogger("")

# reset handler
for handler in logging.root.handlers[:]:
  logging.root.removeHandler(handler)

# set handler
stream_hdlr = logging.StreamHandler()
file_hdlr = logging.FileHandler('/content/gdrive/My Drive/IDL_Project/logs/log_{}.log.log'.format(datetime.datetime.now()))

formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
stream_hdlr.setFormatter(formatter)
file_hdlr.setFormatter(formatter)

logger.addHandler(stream_hdlr)
logger.addHandler(file_hdlr)

logger.setLevel(logging.INFO)

In [11]:
class ShuffleNet(nn.Module):
    def __init__(self, nb_classes =10):
        super(ShuffleNet, self).__init__()
        self.shuffle = models.shufflenet_v2_x2_0()
        self.linear = nn.Linear(1000, nb_classes)
        
    def forward(self, x):
        x = self.shuffle(x)
        x = self.linear(x)
        return x

In [12]:
train_batchsize = 128
test_batchsize = 100
num_workers = 8
num_classes = 10

n_epochs = 25
img_size = 224
lr = 1e-4
min_lr = 1e-8
weight_decay = 5e-4
num_models = 30

noise_std = 0.3
noise_dropout = 0.3

device = torch.device("cuda" if cuda else "cpu")

hyper_params = {'lr': lr, 
                'min_lr': min_lr, 
                'weight_decay': weight_decay, 
                'num_models': num_models, 
                'num_epochs': n_epochs, 
                'noise_std': noise_std, 
                'noise_dropout':noise_dropout}
logging.info(hyper_params)


2020-12-05 03:49:36,642 INFO {'lr': 0.0001, 'min_lr': 1e-08, 'weight_decay': 0.0005, 'num_models': 30, 'num_epochs': 25, 'noise_std': 0.5, 'noise_dropout': 0.5}


In [ ]:
seeds = np.arange(0, num_models)

for seed in seeds:
  logging.info('Seed: {}'.format(seed))
  torch.manual_seed(seed)
  

  train_transform = transforms.Compose([transforms.Resize(size=img_size),
                      transforms.ColorJitter(hue=.05, saturation=.05),
                      transforms.RandomHorizontalFlip(p=0.5),
                      transforms.RandomRotation(20, resample=PIL.Image.BILINEAR),
                      transforms.ToTensor(),
                      transforms.Normalize((0, 0, 0), (1, 1, 1)),
                      AddGaussianNoise(0., noise_std, noise_dropout)])

  test_transform = transforms.Compose([transforms.Resize(size=img_size),
                     transforms.ToTensor(),
                     transforms.Normalize((0, 0, 0), (1, 1, 1)),
                     AddGaussianNoise(0., noise_std, noise_dropout)])

  
  trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
  testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
  trainloader = torch.utils.data.DataLoader(trainset, batch_size=train_batchsize, shuffle=True, num_workers=num_workers)
  testloader = torch.utils.data.DataLoader(testset, batch_size=test_batchsize, shuffle=False, num_workers=num_workers)

  model = ShuffleNet(num_classes)
  model.apply(init_weights)
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
  scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=1, cooldown=5, min_lr=min_lr, verbose=True)
  scaler = torch.cuda.amp.GradScaler()
  model.to(device)

  # Train_loss = []
  # Test_loss = []
  # Test_acc = []

  for i in range(n_epochs):
    tic = time.time()
    logging.info('Epoch: {}'.format(i))
    train_loss = train_epoch(model, trainloader, criterion, optimizer)
    test_loss, test_acc = test_model(model, testloader, criterion)
    scheduler.step(test_loss)
    # Train_loss.append(train_loss)
    # Test_loss.append(test_loss)
    # Test_acc.append(test_acc)
    toc = time.time()
    logging.info('Time: {}s'.format(toc - tic))
    logging.info('='*20)
    
  torch.save({'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict()}, 
        "/content/gdrive/My Drive/IDL_Project/modelS/Model_shufflenet_{}".format(seed))
  
  torch.cuda.empty_cache()
  del model
  del criterion
  del optimizer
  del scheduler
  del scaler
  del trainloader
  del testloader
  del train_transform
  del test_transform

  


2020-12-05 03:50:06,430 INFO Seed: 0


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


2020-12-05 03:50:25,164 INFO Epoch: 0
2020-12-05 03:54:39,073 INFO Training Loss: 2.0350272923784183
2020-12-05 03:54:39,074 INFO Training Accuracy: 24.618000000000002%
2020-12-05 03:55:03,214 INFO Testing Loss: 1.9422382926940918
2020-12-05 03:55:03,215 INFO Testing Accuracy: 31.35%
2020-12-05 03:55:03,216 INFO Time: 278.05252480506897s
2020-12-05 03:55:03,218 INFO ====================
2020-12-05 03:55:03,219 INFO Epoch: 1
2020-12-05 03:59:16,118 INFO Training Loss: 1.7971639547811444
2020-12-05 03:59:16,119 INFO Training Accuracy: 34.086%
2020-12-05 03:59:40,361 INFO Testing Loss: 1.6708365607261657
2020-12-05 03:59:40,362 INFO Testing Accuracy: 38.93%
2020-12-05 03:59:40,364 INFO Time: 277.14510917663574s
2020-12-05 03:59:40,365 INFO ====================
2020-12-05 03:59:40,367 INFO Epoch: 2
2020-12-05 04:03:55,507 INFO Training Loss: 1.6626958715946167
2020-12-05 04:03:55,509 INFO Training Accuracy: 39.058%
2020-12-05 04:04:19,882 INFO Testing Loss: 1.5581510388851165
2020-12-05 04